In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p1_1.csv')
df.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,NaN,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.9,4.98,24.0
1,0.02731,NaN,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.9,9.14,21.6


##### 문제 #1-1. 주어진 Dataset에서 CRIM값이 가장 큰 10개의 지역을 구하고 10개의 지역의 CRIM값을 그 중 가장 작은 값으로 대체하라. 그리고 AGE 컬럼 값이 80이상인 대체 된 CRIM 평균값을 구하라

In [2]:
# 정답
df = df.sort_values('CRIM', ascending=False).reset_index(drop=True)
df.loc[:9, 'CRIM'] = df.loc[:9,'CRIM'].min()
mean = df[df['AGE'] >= 80].CRIM.mean()

In [3]:
print(mean)

5.759386625


##### 문제 #1-2. 1-1에서 사용한 데이터에서 RM 중앙값으로 해당 컬럼의 결측치를 대체하라. 그리고 해당 컬럼의 결측치 대치 전후의 표준편차 차이의 절댓값을 소숫점 이하 3째자리 까지 구하라

In [22]:
prev_std = df['RM'].std()

df['RM'][df['RM'].isna()] = df['RM'].median()
next_std = df['RM'].std()
diff = abs(prev_std-next_std)
print(f'{diff:.3f}')

0.027


C:\Users\cjs98\AppData\Local\Temp\ipykernel_28316\2208731304.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RM'][df['RM'].isna()] = df['RM'].median()


In [ ]:
# 

#### 문제 #1-3. 주어진 Dataset의 DIS 평균으로 부터 1.5 * 표준편차를 벗어나는 영역을 이상치라고 판단하고 DIS 컬럼의 이상치들의 합을 구하여라.

In [24]:
dis_mean, dis_std = df['DIS'].mean(), df['DIS'].std()
upper = dis_mean + 1.5 * dis_std
lower = dis_mean - 1.5 * dis_std
print(df["DIS"][df["DIS"] > upper].sum() + df["DIS"][df["DIS"] < lower].sum())

In [29]:
print(df["DIS"][df["DIS"] > upper].sum() + df["DIS"][df["DIS"] < lower].sum())

404.4101


#### 문제 #2. x_train 데이터로 학습한 모델을 x_test에 적용하여 예측한 결과를 제출하라. 평가 지표는 f1_score이다.

In [163]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")

x_train = x_train.drop(columns=["ID"])
y_train = y_train.drop(columns=["ID"])

x_train["Customer_care_calls"][x_train["Customer_care_calls"]=="$7"] = '7'
x_test["Customer_care_calls"][x_test["Customer_care_calls"]=="$7"] = '7'

imp_map = {'low':0, 'medium':1, "high":2}
x_train['Product_importance'] = x_train["Product_importance"].map(imp_map)
x_test['Product_importance'] = x_test["Product_importance"].map(imp_map)

display(x_train.head())
display(y_train.head())

C:\Users\cjs98\AppData\Local\Temp\ipykernel_28316\312192770.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train["Customer_care_calls"][x_train["Customer_care_calls"]=="$7"] = '7'
C:\Users\cjs98\AppData\Local\Temp\ipykernel_28316\312192770.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test["Customer_care_calls"][x_test["Customer_care_calls"]=="$7"] = '7'


,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,A,Flight,4,3,266,5,2,F,5,1590
1,F,Ship,3,1,174,2,0,M,44,1556
2,F,Road,4,1,154,10,2,M,10,5674
3,F,Ship,4,3,158,3,1,F,27,1207
4,A,Flight,5,3,175,3,0,M,7,4833


,Reached.on.Time_Y.N
0,0
1,1
2,1
3,1
4,1


In [139]:
# x_train["Cost_of_the_Product"].value_counts()
# print(x_train["Discount_offered"].count)

In [165]:
from sklearn.model_selection import train_test_split
x_train = pd.get_dummies(x_train, columns=['Warehouse_block', 'Mode_of_Shipment', 'Gender'])
# x_train = pd.get_dummies(x_train)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
x_train.head()

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Gender_F,Gender_M
3761,4,4,260,2,0,3,5801,0,1,0,0,0,0,0,1,0,1
6099,6,3,269,4,0,4,1948,0,0,1,0,0,1,0,0,1,0
4932,4,5,270,3,1,20,3185,1,0,0,0,0,1,0,0,0,1
4116,5,1,159,2,1,5,4382,0,1,0,0,0,0,0,1,1,0
5752,6,5,254,4,2,10,1217,0,0,0,1,0,0,0,1,1,0


In [194]:
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier, LassoLars
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

In [195]:
clf1 = GaussianNB()

clf1.fit(x_train, y_train)

y_train_pred = clf1.predict(x_train)
y_val_pred = clf1.predict(x_val)

train_f1 = f1_score(y_train_pred, y_train)
val_f1 = f1_score(y_val_pred, y_val)

print(f'train f1 : {train_f1:.4f}')
print(f'val f1: {val_f1:.4f}')

train f1 : 0.5870
val f1: 0.6113


c:\Users\cjs98\anaconda3\envs\gang\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [178]:
clf2 = RidgeClassifier()

clf2.fit(x_train, y_train)

y_train_pred = clf2.predict(x_train)
y_val_pred = clf2.predict(x_val)

train_f1 = f1_score(y_train_pred, y_train)
val_f1 = f1_score(y_val_pred, y_val)

print(f'train f1 : {train_f1:.4f}')
print(f'val f1: {val_f1:.4f}')

train f1 : 0.7097
val f1: 0.7201


c:\Users\cjs98\anaconda3\envs\gang\lib\site-packages\sklearn\linear_model\_ridge.py:1178: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [181]:
clf3 = SVC(
    C=1.0,
    gamma="auto"
)

clf3.fit(x_train, y_train)

y_train_pred = clf3.predict(x_train)
y_val_pred = clf3.predict(x_val)

train_f1 = f1_score(y_train_pred, y_train)
val_f1 = f1_score(y_val_pred, y_val)

print(f'train f1 : {train_f1:.4f}')
print(f'val f1: {val_f1:.4f}')

c:\Users\cjs98\anaconda3\envs\gang\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train f1 : 0.9998
val f1: 0.7528


In [188]:
clf4 = GradientBoostingClassifier(
    loss = 'log_loss',
    n_estimators=10,
    learning_rate=0.01,
    subsample = 0.5,
)
clf4.fit(x_train, y_train)

y_train_pred = clf4.predict(x_train)
y_val_pred = clf4.predict(x_val)

train_f1 = f1_score(y_train_pred, y_train)
val_f1 = f1_score(y_val_pred, y_val)

print(f'train f1 : {train_f1:.4f}')
print(f'val f1: {val_f1:.4f}')

train f1 : 0.7454
val f1: 0.7553


c:\Users\cjs98\anaconda3\envs\gang\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [190]:
clf_hardvot = VotingClassifier(
    estimators=[
        ('ridge', clf2),
        ('svm', clf3),
        ('gb', clf4),
    ]
    ,
    voting = 'hard'
)

clf_hardvot.fit(x_train, y_train)

y_train_pred = clf_hardvot.predict(x_train)
y_val_pred = clf_hardvot.predict(x_val)

train_f1 = f1_score(y_train_pred, y_train)
val_f1 = f1_score(y_val_pred, y_val)

print(f'train f1 : {train_f1:.4f}')
print(f'val f1: {val_f1:.4f}')

c:\Users\cjs98\anaconda3\envs\gang\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cjs98\anaconda3\envs\gang\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train f1 : 0.8729
val f1: 0.7530


In [196]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p3_1.csv')
df.head(5)

,fish,height
0,fish_1,33.74cm
1,fish_2,31.64cm
2,fish_3,27.49cm
3,fish_4,31.43cm
4,fish_5,27.56cm


#### 문제 #3-1-a. 122마리의 height 평균값을 m(미터) 단위로 소숫점 이하 5자리까지 실수 값만 출력하라.

#### 문제 #3-1-b. 모집단의 평균 길이가 30cm 인지 확인하려 일표본 t 검정을 시행하여 확인하려한다. 검정통계량을 소숫점 이하 3째자리까지 구하여라

#### 문제 #3-1-c. 위의 통계량에 대한 p-값을 구하고 (반올림하여 소숫점 이하 3째자리), 유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)

#### 문제 #3-2-a. 위의 통계량에 대한 p-값을 구하고 (반올림하여 소숫점 이하 3째자리), 유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)

#### 문제 #3-2-b. 적어도 19명이 치과를 찾았을 확률(반올림하여 소숫점 이하 3자리)